# Práctica ETL: Histograma y Ejercicios Examen

### Autor: Lucía Saiz Lapique

### Enunciado: 
Vamos a poner en práctica lo que hemos visto de spark. Para ellos vamos a trabajar con la siguiente fuente de datos: partidosLigaNBA. El enunciado del ejercicio lo tenéis a continuación. 

Pasos: 

1 Parsear el dataset para su tratamiento (lo que hemos visto hasta ahora, lo necesario para poder usar la información que se nos pide)

2 Breve análisis descriptivo para analizar si el dataset está límpio

3 Enunciado

4 Generar histograma de los puntos anotados por los visitantes (este último punto será para la semana siguiente)

Debéis subir el código, en .py o en .ipynb (sin los datos resultantes que si puede ocupar mucho). El objetivo es aplicar map-reduce, veremos más adelante como se puede hacer de manera más sencilla con sql. Se evaluaran tanto resultados con optimización en el uso.º

NOTA: A los que no os funciona docker o spark en vuestro ordenador, podéis hacer el resto de tareas. Con respecto a esta y la siguiente podéis ir desarrollando el código usando jupyter en anaconda.

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de partidosLigaNBA.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

## Primera pregunta: Describe brevemente que diferencia el persists, cache y collect en spark. Explica brevemente casos en los que es interesante su aplicación

Persists y cache son técnicas de optimización para computaciones realizadas en Spark. Se utilizan para almacenar los datos y la única diferencia entre ellos es que cache usa el nivel de MEMORY_ONLY por defecto y con persists puedes seleccionar dónde quieres almacenar las cosas. El uso de uno u otro es prácticamente el mismo, la única ocasión en la que elegiríamos entre uno de ellos es si no queremos usar la memoria de defecto para almacenar los datos seleccionados.

Collect es una función distinta, que lleva todos los datos seleccionados del dataset como un array al programa; es útil después de haber ejecutado algún filtro o tras haber reducido el tamaño del datsset y pasarlo a un subset. Nos devuelve el resultado de todos los datos que le pedimos.

## Segunda pregunta: Explica brevemente los pasos a seguir para realizar la carga de un conjunto de datos (pasos que se siguieron en la práctica con datos de logs)

El primer paso para cargar un conjunto de datos que queramos usar en un notebook de Python 3 crear un nuevo Jupyter Notebook y tener la base de datos (ya sea en formato csv, json o yalm) descargada en el mismo directorio en el que está guardado el notebook que se va a utilizar. En caso de querer utilizar una base de datos que se encuentre en una página web, podremos también usarla sin necesidad de decargar y guardarla en nuestro directorio.

En segundo lugar, debemos importar el paquete o librería que queramos ejecutar (ya sea pandas en caso de trabajar con pandas o pyspark si trabajamos con Docker) para obtener acceso a las funciones necesarias. En nuestro caso específico, utilizaremos la librería pyspark y un fichero de datos que tenemos descargado con formato csv. Para importar el paquete en cuestión escribimos lo siguiente en el Notebook:

from pyspark import SparkContext

sc = SparkContext("local", "First App")

Cuando eso está ejecutado, creamos un nuevo objeto en el que vayamos a guardar la base de datos. Al objeto le ponemos un nombre facil y corto, ya que vamos a estar usando a lo largo de todo el ejercicio. En nuestro caso, aplicamos también la función textFile para que python nos lea los datos y los devuelva como un string de RDD. 

## Tercera Pregunta: Índica un tipo de problema que puede empeorar los datos. (pe. Que no exista un representante del CDO en todas las áreas de negocio), pon algún ejemplo específico (pe. Datos duplicados) y cómo lo tratarías con técnicas de data cleaning.

Un problema muy común que puede surgir al importar una base de datos en python y que puede empeorar los datos es que los datos estén introducidos erróneamente; por ejemplo, que un dato que sea igual para varias observaciones esté escrito de distinta forma (si piden profesión y por ejemplo ponen maestro, Profesor, profesora, Maestra, etc.). Es muy difícil interpretar datos y buscar similitudes entre observaciones si lo siguen todos la misma métrica o formato. 

Para resolverlo habría que buscar todos los datos distintos que hay de las variables y asociar a cada uno que sea distinto pero del mismo tipo, un nombre común. 

## Cuarta tarea: Inicializar spark context y cargar los datos desde el fichero.

In [2]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")

In [3]:
nba = "./partidosLigaNBA.csv"
nba = sc.textFile(nba)

In [4]:
nba

./partidosLigaNBA.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

### Parseamos los datos

In [5]:
# para quedarme sin el header y poder trabajar con los datos directamente; parseamos separando los datos con el símbolo ":".
nba = nba.filter(lambda x: "PTS" not in x).map(lambda x: x.split(":"))

In [6]:
nba.take(1)  # llamamos a la primera línea de datos del dataset para comprobar que está todo correctamente separado.
# vemos que contamos con la fecha de cada partido, la hora a la que se jugó, el nombre y puntos del equipo visitante y el del 
# grupo de casa. 

[['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96']]

### Análisis descriptivo de los datos

In [7]:
nba.count() # primero identificamos el número de observaciones con el que contamos

12907

In [8]:
nba = nba.filter(lambda x: "Playoffs" not in x) # hacemos un filtro en el que quitemos las palabras Playoffs

In [9]:
nba.take(1)

[['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96']]

Para análizar de manera general los datos, sacamos la media, desviación típica, máximos y mínimos de los puntos de visitantes y de casa. 

In [10]:
media_visit = round(nba.map(lambda x: int(x[4])).mean(),2)
media_casa = round(nba.map(lambda x: int(x[6])).mean(),2)
print("La media de puntos de los visitantes es", media_visit, "y la de los de casa es", media_casa)

La media de puntos de los visitantes es 98.74 y la de los de casa es 101.78


In [11]:
des_visit = round(nba.map(lambda x: int(x[4])).stdev(),2)
des_casa = round(nba.map(lambda x: int(x[6])).stdev(),2)
print("La desviación típica de puntos de los visitantes es", des_visit, "y la de los de casa es", des_casa)

La desviación típica de puntos de los visitantes es 12.13 y la de los de casa es 12.26


In [12]:
max_visit = nba.map(lambda x: int(x[4])).max()
max_casa = nba.map(lambda x: int(x[6])).max()
print("El valor máximo de los puntos de los visitantes es", max_visit, "y el de los de casa es", max_casa)

El valor máximo de los puntos de los visitantes es 154 y el de los de casa es 168


In [13]:
min_visit = nba.map(lambda x: int(x[4])).min()
min_casa = nba.map(lambda x: int(x[6])).min()
print("El valor mínimo de los puntos de los visitantes es", min_visit, "y el de los de casa es", min_casa)

El valor mínimo de los puntos de los visitantes es 54 y el de los de casa es 59


## Quinta tarea: Media de la diferencia de puntos por año

Para averiguar cual es la media de la diferencia de puntos por año debemos llevar a cabo una serie de pasos. En primer lugar, le asociaremos todos los puntos de visita y de casa que se marcaron por temporada a cada año, por separado. Por un lado tendremos una lista con una tupla por año que esté formada por dos elementos; el año y los puntos de visitante ese año. Por otro lado tendremos el mismo tipo de lista con los puntos adquiridos en casa. Una vez tenemos las dos listas con los puntos de visitante y de casa, buscamos la diferencia entre ellos y se la asociamos a cada año.

In [14]:
fecha = nba.map(lambda x: x[0][-4:]) # nos piden trabajar con el año

In [15]:
ptos_visita = nba.map(lambda x: int(x[4]))
ptos_visita.take(10)

[117, 95, 97, 92, 103, 110, 104, 103, 90, 83]

In [16]:
ptos_casa = nba.map(lambda x: int(x[6]))
ptos_casa.take(5)

[96, 93, 106, 74, 120]

In [107]:
ptos_fecha_visit = nba.map(lambda x: (x[0][-4:], int(x[4])))
ptos_fecha_visit_key = ptos_fecha_visit.reduceByKey(lambda x, y: x + y)
ptos_fecha_visit_key.take(5)

[('2017', 84102),
 ('2013', 128963),
 ('2016', 136221),
 ('2011', 86293),
 ('2014', 133100)]

In [18]:
ptos_fecha_casa = nba.map(lambda x: (x[0][-4:], int(x[6])))
ptos_fecha_casa_key = ptos_fecha_casa.reduceByKey(lambda x, y: x + y)
ptos_fecha_casa_key.take(5)

[('2017', 86682),
 ('2013', 132642),
 ('2016', 140466),
 ('2011', 88998),
 ('2014', 136567)]

In [19]:
ptos_fecha = nba.map(lambda x: (x[0][-4:], abs(int(x[4])-int(x[6]))))
ptos_fecha_key = ptos_fecha.reduceByKey(lambda x, y: x + y)
ptos_fecha_key.take(5)

[('2017', 9172),
 ('2013', 14659),
 ('2016', 15397),
 ('2011', 9435),
 ('2014', 14547)]

El siguiente paso, una vez tenemos los puntos totales adquiridos por año, es averiguar cuántos partidos se han jugado en esas temporadas, para después calcular la media. Para ver el número de partidos que se han jugado, hacemos un conteo asociándole un 1 a cada observación que hay según su año. Con ello obtenemos el número total de partidos y podemos calcular la media. 

In [20]:
partidos = nba.map(lambda x: (x[0][-4:], 1))
partidos_key = partidos.reduceByKey(lambda x, y: x + y)
partidos_key.take(5)  

[('2017', 803), ('2013', 1324), ('2016', 1333), ('2011', 885), ('2014', 1334)]

In [21]:
ptos_partidos = ptos_fecha_key.join(partidos_key)  # join nos junta los puntos y los partidos asociados a cada año en base al 
# valor común que tienen; la fecha
ptos_partidos.take(5)

[('2017', (9172, 803)),
 ('2013', (14659, 1324)),
 ('2016', (15397, 1333)),
 ('2011', (9435, 885)),
 ('2014', (14547, 1334))]

In [22]:
media = ptos_partidos.map(lambda x: (x[0], round(int(x[1][0])/int(x[1][1]), 2))).sortBy(lambda x: x[0])
media.collect()
# a continuación, una lista con la media de diferencia de puntos por partido en cada temporada. 

[('2007', 11.1),
 ('2008', 11.54),
 ('2009', 11.09),
 ('2010', 10.87),
 ('2011', 10.66),
 ('2012', 10.85),
 ('2013', 11.07),
 ('2014', 10.9),
 ('2015', 11.16),
 ('2016', 11.55),
 ('2017', 11.42)]

## Sexta tarea: ¿Han judado todos los equipos el mismo número de partidos? ¿ Si es qué no a que puede deberse?

En este caso tenemos que, como en el caso anterior, hacer un conteo para ver cuántos partidos ha jugado cada partido en todas las temporadas. Hay que tener en cuenta los partidos de visitas y los de casa, así que realizamos el conteo sobre el nombre de  cad aequipo, primero de los partidos en los que jugaban como visitantes y luego en los que jugaban en casa, para después con un join, unirlos y sumar el total. 

In [23]:
partidos_visita = nba.map(lambda x: (x[3], 1))
partidos_visita_key = partidos_visita.reduceByKey(lambda x, y: x + y)
partidos_visita_key.take(5)

[('New Orleans Pelicans', 166),
 ('Sacramento Kings', 402),
 ('Charlotte Hornets', 127),
 ('Phoenix Suns', 413),
 ('San Antonio Spurs', 466)]

In [24]:
partidos_casa = nba.map(lambda x: (x[5], 1))
partidos_casa_key = partidos_casa.reduceByKey(lambda x, y: x + y)
partidos_casa_key.take(5)

[('New Orleans Pelicans', 166),
 ('Sacramento Kings', 402),
 ('Atlanta Hawks', 448),
 ('Charlotte Hornets', 126),
 ('Phoenix Suns', 412)]

In [25]:
partidos_ambos = partidos_visita_key.join(partidos_casa_key)
partidos_ambos.take(5)

[('New Orleans Pelicans', (166, 166)),
 ('Charlotte Hornets', (127, 126)),
 ('Phoenix Suns', (413, 412)),
 ('New Jersey Nets', (197, 197)),
 ('Golden State Warriors', (440, 445))]

In [26]:
partidos_total = partidos_ambos.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1])))).sortBy(lambda x: -x[1])
partidos_total.collect()

[('San Antonio Spurs', 933),
 ('Boston Celtics', 930),
 ('Miami Heat', 917),
 ('Cleveland Cavaliers', 901),
 ('Los Angeles Lakers', 897),
 ('Atlanta Hawks', 896),
 ('Golden State Warriors', 885),
 ('Chicago Bulls', 873),
 ('Indiana Pacers', 868),
 ('Houston Rockets', 868),
 ('Dallas Mavericks', 867),
 ('Memphis Grizzlies', 867),
 ('Orlando Magic', 863),
 ('Los Angeles Clippers', 861),
 ('Portland Trail Blazers', 853),
 ('Toronto Raptors', 850),
 ('Denver Nuggets', 848),
 ('Utah Jazz', 846),
 ('Washington Wizards', 844),
 ('Philadelphia 76ers', 834),
 ('Detroit Pistons', 829),
 ('Milwaukee Bucks', 827),
 ('Phoenix Suns', 825),
 ('New York Knicks', 825),
 ('Oklahoma City Thunder', 818),
 ('Sacramento Kings', 804),
 ('Minnesota Timberwolves', 804),
 ('Charlotte Bobcats', 566),
 ('New Orleans Hornets', 499),
 ('Brooklyn Nets', 435),
 ('New Jersey Nets', 394),
 ('New Orleans Pelicans', 332),
 ('Charlotte Hornets', 253),
 ('Seattle SuperSonics', 82)]

Al imprimir la lista final al completo, vemos como el equipo que más partidos ha jugado ha sido los San Antonio Spurs con un total de partidos jugados de 933. La razón por la que unos equipos juegan más partidos que otros puede ser que hayan cambiado de nombre y nen el dataset salgan los partidos por separado o que, al ser partidos de "playoffs", los que pierden mucho se van eliminandi y, por tanto, juegan menos que los mejores equipos. Podemos concluir pues, que los mejores equipos son los que más partidos han jugado.

## Séptima pregunta: ¿Cuántos partidos ha ganado en enero Cleveland?

Para calcular el número de partidos que gana en enero Cleveland, tenemos que coger nuestra base de datos y aplicarle dos filtros: uno que recoja los datos de todos los partidos que se han jugado en todas las temporadas en enero y otro que incluya, de esos, solo los que juegan los Cleveland Cavaliers, tanto en visita como en casa. 

In [27]:
nba.take(1)

[['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96']]

In [28]:
nba.map(lambda x: x[0][5:8]).distinct().collect() # sacamos los meses para buscar cuantos caracteres tiene enero y como se escribe,
# pues lo necesitamos para el filtro. 

['Apr', 'Jan', 'Nov', 'Jun', 'Mar', 'Oct', 'Dec', 'May', 'Feb']

In [29]:
partidos_enero = nba.filter(lambda x: x[0][5:8] == 'Jan')  #  el filter recorre todo el dataset, como el map
partidos_enero.take(1)

[['Wed, Jan 2, 2008',
  '"7',
  '30 pm"',
  'Houston Rockets',
  '93',
  'Boston Celtics',
  '97']]

In [30]:
nba.map(lambda x: x[3]).distinct().collect() # buscamos el nombre exacto del equipo para hacer el filtro como con enero

['New Orleans Pelicans',
 'Sacramento Kings',
 'Charlotte Hornets',
 'Phoenix Suns',
 'San Antonio Spurs',
 'Miami Heat',
 'Golden State Warriors',
 'Brooklyn Nets',
 'Milwaukee Bucks',
 'New Jersey Nets',
 'Seattle SuperSonics',
 'Utah Jazz',
 'Los Angeles Lakers',
 'Indiana Pacers',
 'Houston Rockets',
 'Dallas Mavericks',
 'Oklahoma City Thunder',
 'Washington Wizards',
 'Cleveland Cavaliers',
 'Chicago Bulls',
 'Toronto Raptors',
 'Portland Trail Blazers',
 'New Orleans Hornets',
 'Memphis Grizzlies',
 'Denver Nuggets',
 'Philadelphia 76ers',
 'New York Knicks',
 'Orlando Magic',
 'Boston Celtics',
 'Atlanta Hawks',
 'Detroit Pistons',
 'Minnesota Timberwolves',
 'Charlotte Bobcats',
 'Los Angeles Clippers']

In [31]:
partidos_clev_visita = partidos_enero.filter(lambda x: x[3] == 'Cleveland Cavaliers')  
partidos_clev_visita.take(1)

[['Sun, Jan 6, 2008',
  '"1',
  '00 pm"',
  'Cleveland Cavaliers',
  '93',
  'Toronto Raptors',
  '90']]

In [32]:
ganados_visit = partidos_clev_visita.filter(lambda x: int(x[4]) > int(x[6]))
ganados_visit_num = ganados_visit.count()

In [33]:
partidos_clev_casa = partidos_enero.filter(lambda x: x[5] == 'Cleveland Cavaliers')  #  el filter tb recorre todo
partidos_clev_casa.take(1)

[['Wed, Jan 2, 2008',
  '"7',
  '00 pm"',
  'Atlanta Hawks',
  '94',
  'Cleveland Cavaliers',
  '98']]

In [34]:
ganados_casa = partidos_clev_casa.filter(lambda x: int(x[6]) > int(x[4]))
ganados_casa_num = ganados_casa.count()

In [35]:
ganados_total = ganados_casa_num + ganados_visit_num
print("Los Cleveland Cavaliers han ganado en enero", ganados_total, "partidos.")

Los Cleveland Cavaliers han ganado en enero 83 partidos.


## Octava pregunta: ¿Los Warriors son mejores fuera de casa o en casa?

Para averiguar donde juegan mejor los warriors, si en casa o fuera, usamos filtros de forma parecida al ejercicio anterior. En este caso no necesitaríamos filtrar por fecha. 

In [36]:
partidos_warriors_visita = nba.filter(lambda x: x[3] == 'Golden State Warriors')
partidos_warriors_visita.take(3)

[['Fri, Nov 2, 2007',
  '"7',
  '30 pm"',
  'Golden State Warriors',
  '114',
  'Los Angeles Clippers',
  '120'],
 ['Sat, Nov 3, 2007',
  '"7',
  '00 pm"',
  'Golden State Warriors',
  '110',
  'Utah Jazz',
  '133'],
 ['Sun, Nov 18, 2007',
  '"1',
  '00 pm"',
  'Golden State Warriors',
  '106',
  'Toronto Raptors',
  '100']]

In [37]:
ganados_visit_war = partidos_warriors_visita.filter(lambda x: int(x[4]) > int(x[6]))
ganados_visit_war.count()

215

In [38]:
partidos_warriors_casa = nba.filter(lambda x: x[5] == 'Golden State Warriors')
partidos_warriors_casa.take(3)

[['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96'],
 ['Tue, Nov 6, 2007',
  '"7',
  '30 pm"',
  'Cleveland Cavaliers',
  '108',
  'Golden State Warriors',
  '104'],
 ['Thu, Nov 8, 2007',
  '"7',
  '30 pm"',
  'Dallas Mavericks',
  '120',
  'Golden State Warriors',
  '115']]

In [39]:
ganados_casa_war = partidos_warriors_casa.filter(lambda x: int(x[6]) > int(x[4]))
ganados_casa_war.count()

308

In [40]:
ganados_war = max(ganados_visit_war.count(), ganados_casa_war.count())
ganados_war

308

In [41]:
def warriors():
    if (ganados_casa_war.count() > ganados_visit_war.count()):
        print("El equipo de los Golden State Warriors ha ganado más partidos en casa, con un total de", ganados_casa_war.count(), "partidos.")
    else:
        print("El equipo de los Golden State Warriors ha ganado más partidos en casa, con un total de", ganados_visit_war.count(), "partidos.")       
warriors()

El equipo de los Golden State Warriors ha ganado más partidos en casa, con un total de 308 partidos.


## Novena pregunta: Equipo que ha quedado primerio en victorias más temporadas. (si es que hay alguno que más)

Este es el ejercicio más largo. en este ejercicio hay que averiguar cuantas victorias tiene cada equipo tanto en casa como de visitantes en todos los años, y luego averiguar qué equipo tiene más victorias de todos ellos. 

In [42]:
nba.take(1)

[['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96']]

In [43]:
ganados_visit_total = nba.filter(lambda x: int(x[4]) > int(x[6]))
ganados_visit_total.take(1)

[['Tue, Oct 30, 2007',
  '"7',
  '30 pm"',
  'Utah Jazz',
  '117',
  'Golden State Warriors',
  '96']]

In [44]:
ganados_casa_total = nba.filter(lambda x: int(x[6]) > int(x[4]))
ganados_casa_total.take(1)

[['Tue, Oct 30, 2007',
  '"7',
  '00 pm"',
  'Portland Trail Blazers',
  '97',
  'San Antonio Spurs',
  '106']]

In [45]:
nba.map(lambda x: x[0][-4:]).distinct().collect()

['2017',
 '2013',
 '2016',
 '2011',
 '2014',
 '2008',
 '2007',
 '2015',
 '2009',
 '2010',
 '2012']

Buscamos los equipos que hayan ganado el mayor número de partidos en 2007.

In [46]:
ganados_2007_visit = ganados_visit_total.filter(lambda x: x[0][-4:] == '2007')
ganados_2007_casa = ganados_casa_total.filter(lambda x: x[0][-4:] == '2007')

In [47]:
visita_2007 = ganados_2007_visit.map(lambda x: (x[3], 1))
visita_2007_key = visita_2007.reduceByKey(lambda x, y: x + y)
casa_2007 = ganados_2007_casa.map(lambda x: (x[5], 1))
casa_2007_key = casa_2007.reduceByKey(lambda x, y: x + y)
juntos_2007 = visita_2007_key.join(casa_2007_key)
juntos_2007.take(5)

[('Minnesota Timberwolves', (1, 3)),
 ('Utah Jazz', (6, 11)),
 ('Phoenix Suns', (13, 9)),
 ('New Jersey Nets', (7, 7)),
 ('Golden State Warriors', (11, 8))]

In [48]:
yr_07 = juntos_2007.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1])))).sortBy(lambda x: -x[1])
yr_07.take(3)

[('Boston Celtics', 26), ('Detroit Pistons', 24), ('Phoenix Suns', 22)]

Buscamos los equipos que hayan ganado el mayor número de partidos en 2008.

In [49]:
ganados_2008_visit = ganados_visit_total.filter(lambda x: x[0][-4:] == '2008')
ganados_2008_casa = ganados_casa_total.filter(lambda x: x[0][-4:] == '2008')

In [50]:
visita_2008 = ganados_2008_visit.map(lambda x: (x[3], 1))
visita_2008_key = visita_2008.reduceByKey(lambda x, y: x + y)
casa_2008 = ganados_2008_casa.map(lambda x: (x[5], 1))
casa_2008_key = casa_2008.reduceByKey(lambda x, y: x + y)
juntos_2008 = visita_2008_key.join(casa_2008_key)

In [51]:
yr_08 = juntos_2008.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1])))).sortBy(lambda x: -x[1])
yr_08.take(3)

[('Boston Celtics', 84), ('Los Angeles Lakers', 77), ('San Antonio Spurs', 64)]

Equipos que hayan ganado el mayor número de partidos en 2009.

In [52]:
ganados_2009_visit = ganados_visit_total.filter(lambda x: x[0][-4:] == '2009')
ganados_2009_casa = ganados_casa_total.filter(lambda x: x[0][-4:] == '2009')
visita_2009 = ganados_2009_visit.map(lambda x: (x[3], 1))
visita_2009_key = visita_2009.reduceByKey(lambda x, y: x + y)
casa_2009 = ganados_2009_casa.map(lambda x: (x[5], 1))
casa_2009_key = casa_2009.reduceByKey(lambda x, y: x + y)
juntos_2009 = visita_2009_key.join(casa_2008_key)
yr_09 = juntos_2009.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1])))).sortBy(lambda x: -x[1])
yr_09.take(3)

[('Boston Celtics', 82),
 ('Los Angeles Lakers', 79),
 ('Cleveland Cavaliers', 74)]

Equipos que hayan ganado el mayor número de partidos en 2010.

In [53]:
ganados_2010_visit = ganados_visit_total.filter(lambda x: x[0][-4:] == '2010')
ganados_2010_casa = ganados_casa_total.filter(lambda x: x[0][-4:] == '2010')
visita_2010 = ganados_2010_visit.map(lambda x: (x[3], 1))
visita_2010_key = visita_2010.reduceByKey(lambda x, y: x + y)
casa_2010 = ganados_2010_casa.map(lambda x: (x[5], 1))
casa_2010_key = casa_2010.reduceByKey(lambda x, y: x + y)
juntos_2010 = visita_2010_key.join(casa_2010_key)
yr_10 = juntos_2010.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1])))).sortBy(lambda x: -x[1])
yr_10.take(3)

[('Los Angeles Lakers', 71), ('Orlando Magic', 67), ('Boston Celtics', 66)]

Equipos que hayan ganado el mayor número de partidos en 2011.

In [54]:
ganados_2011_visit = ganados_visit_total.filter(lambda x: x[0][-4:] == '2011')
ganados_2011_casa = ganados_casa_total.filter(lambda x: x[0][-4:] == '2011')
visita_2011 = ganados_2011_visit.map(lambda x: (x[3], 1))
visita_2011_key = visita_2011.reduceByKey(lambda x, y: x + y)
casa_2011 = ganados_2011_casa.map(lambda x: (x[5], 1))
casa_2011_key = casa_2011.reduceByKey(lambda x, y: x + y)
juntos_2011 = visita_2011_key.join(casa_2011_key)
yr_11 = juntos_2011.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1])))).sortBy(lambda x: -x[1])
yr_11.take(3)

[('Chicago Bulls', 53), ('Miami Heat', 51), ('Dallas Mavericks', 50)]

Equipos que hayan ganado el mayor número de partidos en 2012.

In [55]:
ganados_2012_visit = ganados_visit_total.filter(lambda x: x[0][-4:] == '2012')
ganados_2012_casa = ganados_casa_total.filter(lambda x: x[0][-4:] == '2012')
visita_2012 = ganados_2012_visit.map(lambda x: (x[3], 1))
visita_2012_key = visita_2012.reduceByKey(lambda x, y: x + y)
casa_2012 = ganados_2012_casa.map(lambda x: (x[5], 1))
casa_2012_key = casa_2012.reduceByKey(lambda x, y: x + y)
juntos_2012 = visita_2012_key.join(casa_2012_key)
yr_12 = juntos_2012.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1])))).sortBy(lambda x: -x[1])
yr_12.take(3)

[('San Antonio Spurs', 82), ('Miami Heat', 79), ('Oklahoma City Thunder', 79)]

Equipos que hayan ganado el mayor número de partidos en 2013.

In [56]:
ganados_2013_visit = ganados_visit_total.filter(lambda x: x[0][-4:] == '2013')
ganados_2013_casa = ganados_casa_total.filter(lambda x: x[0][-4:] == '2013')
visita_2013 = ganados_2013_visit.map(lambda x: (x[3], 1))
visita_2013_key = visita_2013.reduceByKey(lambda x, y: x + y)
casa_2013 = ganados_2013_casa.map(lambda x: (x[5], 1))
casa_2013_key = casa_2013.reduceByKey(lambda x, y: x + y)
juntos_2013 = visita_2013_key.join(casa_2013_key)
yr_13 = juntos_2013.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1])))).sortBy(lambda x: -x[1])
yr_13.take(3)

[('Miami Heat', 85), ('San Antonio Spurs', 73), ('Indiana Pacers', 67)]

Equipos que hayan ganado el mayor número de partidos en 2014.

In [57]:
ganados_2014_visit = ganados_visit_total.filter(lambda x: x[0][-4:] == '2014')
ganados_2014_casa = ganados_casa_total.filter(lambda x: x[0][-4:] == '2014')
visita_2014 = ganados_2014_visit.map(lambda x: (x[3], 1))
visita_2014_key = visita_2014.reduceByKey(lambda x, y: x + y)
casa_2014 = ganados_2014_casa.map(lambda x: (x[5], 1))
casa_2014_key = casa_2014.reduceByKey(lambda x, y: x + y)
juntos_2014 = visita_2014_key.join(casa_2014_key)
yr_14 = juntos_2014.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1])))).sortBy(lambda x: -x[1])
yr_14.take(3)

[('San Antonio Spurs', 73),
 ('Los Angeles Clippers', 64),
 ('Memphis Grizzlies', 63)]

Equipos que hayan ganado el mayor número de partidos en 2015.

In [58]:
ganados_2015_visit = ganados_visit_total.filter(lambda x: x[0][-4:] == '2015')
ganados_2015_casa = ganados_casa_total.filter(lambda x: x[0][-4:] == '2015')
visita_2015 = ganados_2015_visit.map(lambda x: (x[3], 1))
visita_2015_key = visita_2015.reduceByKey(lambda x, y: x + y)
casa_2015 = ganados_2015_casa.map(lambda x: (x[5], 1))
casa_2015_key = casa_2015.reduceByKey(lambda x, y: x + y)
juntos_2015 = visita_2015_key.join(casa_2015_key)
yr_15 = juntos_2015.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1])))).sortBy(lambda x: -x[1])
yr_15.take(3)

[('Golden State Warriors', 88),
 ('Cleveland Cavaliers', 70),
 ('San Antonio Spurs', 66)]

Equipos que hayan ganado el mayor número de partidos en 2016.

In [59]:
ganados_2016_visit = ganados_visit_total.filter(lambda x: x[0][-4:] == '2016')
ganados_2016_casa = ganados_casa_total.filter(lambda x: x[0][-4:] == '2016')
visita_2016= ganados_2016_visit.map(lambda x: (x[3], 1))
visita_2016_key = visita_2016.reduceByKey(lambda x, y: x + y)
casa_2016 = ganados_2016_casa.map(lambda x: (x[5], 1))
casa_2016_key = casa_2016.reduceByKey(lambda x, y: x + y)
juntos_2016 = visita_2016_key.join(casa_2016_key)
yr_16 = juntos_2016.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1])))).sortBy(lambda x: -x[1])
yr_16.take(3)

[('Golden State Warriors', 87),
 ('Cleveland Cavaliers', 77),
 ('San Antonio Spurs', 72)]

Equipos que hayan ganado el mayor número de partidos en 2017.

In [60]:
ganados_2017_visit = ganados_visit_total.filter(lambda x: x[0][-4:] == '2017')
ganados_2017_casa = ganados_casa_total.filter(lambda x: x[0][-4:] == '2017')
visita_2017 = ganados_2017_visit.map(lambda x: (x[3], 1))
visita_2017_key = visita_2017.reduceByKey(lambda x, y: x + y)
casa_2017 = ganados_2017_casa.map(lambda x: (x[5], 1))
casa_2017_key = casa_2017.reduceByKey(lambda x, y: x + y)
juntos_2017 = visita_2017_key.join(casa_2017_key)
yr_17 = juntos_2017.map(lambda x: (x[0], (int(x[1][0]) + int(x[1][1])))).sortBy(lambda x: -x[1])
yr_17.take(3)

[('Golden State Warriors', 54),
 ('San Antonio Spurs', 42),
 ('Boston Celtics', 42)]

In [61]:
mejores_temp = [('2007', yr_07.take(1)[0][0]), ('2008', yr_08.take(1)[0][0]), ('2009', yr_09.take(1)[0][0]), ('2010', yr_10.take(1)[0][0]), 
                ('2011', yr_11.take(1)[0][0]), ('2012', yr_12.take(1)[0][0]), ('2013', yr_13.take(1)[0][0]), ('2014', yr_14.take(1)[0][0]),
               ('2015', yr_15.take(1)[0][0]), ('2016', yr_16.take(1)[0][0]), ('2016', yr_16.take(1)[0][0])]
mejores_temp

[('2007', 'Boston Celtics'),
 ('2008', 'Boston Celtics'),
 ('2009', 'Boston Celtics'),
 ('2010', 'Los Angeles Lakers'),
 ('2011', 'Chicago Bulls'),
 ('2012', 'San Antonio Spurs'),
 ('2013', 'Miami Heat'),
 ('2014', 'San Antonio Spurs'),
 ('2015', 'Golden State Warriors'),
 ('2016', 'Golden State Warriors'),
 ('2016', 'Golden State Warriors')]

Los equipos que más victorias han obtenido en más temporadas son los Boston Celtics y los Golden State Warriors. 

##  Décima pregunta: Escribe la expresión regular correcta que sólo macheen los teléfonos y el correo del siguiente texto.

Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com

In [62]:
import re

In [63]:
texto = "Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com"

In [64]:
exp_telf = '\d{3} \d{2} \d{2} \d{2}'
exp_corr = '\S+[@]\S+[.com]'

In [65]:
re.findall(exp_telf, texto)

['902 13 23 13', '902 88 30 08', '902 88 88 35']

In [66]:
re.findall(exp_corr, texto)

['atencioncliente@bankinter.com']

In [7]:
sc.stop() # necesario para cerrar pyspark y usarlo en otro sitio

## BIBLIOGRAFÍA

* https://jaceklaskowski.gitbooks.io/mastering-apache-spark/spark-rdd-caching.html
* https://stackoverflow.com/questions/44174747/spark-dataframe-collect-vs-select
* https://empresas.blogthinkbig.com/python-para-todos-4-carga-de-datos/